In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
movies=pd.read_csv("/content/drive/MyDrive/adverk/datasets/tmdb_5000_movies.csv")
credits=pd.read_csv("/content/drive/MyDrive/adverk/datasets/tmdb_5000_credits.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
movies.head(1)

In [ ]:
movies.shape

In [ ]:
credits.head(1)

In [ ]:
credits.shape

In [ ]:
movies=movies.merge(credits,left_on="id",right_on="movie_id")

In [ ]:
movies.shape

In [ ]:
movies.head(1)

In [ ]:
movies.columns

In [ ]:
# genres,id,keywords,title_x,overview,cast,crew

In [ ]:
movies=movies[['movie_id','title_x','overview','genres','keywords','cast','crew']]

In [ ]:
movies.shape

In [ ]:
movies.head(1)

In [ ]:
movies.rename(columns={'title_x':'title'},inplace=True)

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies['genres']

In [ ]:
movies.sample()

In [ ]:
a=movies.iloc[1918].genres
a

In [ ]:
type(a)

In [ ]:
x={"id": 28, "name": "Action"}

In [ ]:
x['id']

In [ ]:
#ast.literal_eval("{'a':'b','c':'d','e':'f'}")

In [ ]:
ast.literal_eval("[2,4,6,7]")

In [ ]:
import ast
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [ ]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 35, "name": "Comedy"}, {"id": 14, "name": "Fantasy"}]')

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies['genres']

In [ ]:
movies.head(3)

In [ ]:
movies.iloc[0].keywords

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.sample(4)

In [ ]:
movies.iloc[1].cast

In [ ]:
def convert3(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter !=3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=="Director":
      L.append(i['name'])
      break
  return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head(10)

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [ ]:
movies.head(10)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(3)

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.sample()

In [ ]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][8]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
s=['Congrats. You have won a lottery and you can get lottery amount by calling the lottery number',
   'Give your bank account details for lottery amount to be credited to your bank account',
   'lottery for sure if the bank account details are verified']

In [ ]:
s[0].split()  # Tokenization

In [ ]:
s[1].split()

In [ ]:
s[2].split()

In [ ]:
vect=CountVectorizer()
op=vect.fit_transform(s).toarray()
op

In [ ]:
vect.get_feature_names_out()

In [ ]:
import pandas as pd
pd.DataFrame(op,columns=vect.get_feature_names_out(),index=['s[1]','s[2]','s[3]'])

In [ ]:
# stop_words
# Stop words are the words in a stop list which are filtered out before or after processing of natural language data because they are insignificant.
# Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence.

In [ ]:
vect=CountVectorizer(stop_words='english')
op=vect.fit_transform(s).toarray()
pd.DataFrame(op,columns=vect.get_feature_names_out(),index=['s[1]','s[2]','s[3]'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english',max_features=5000)

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names_out()[0:200]

In [ ]:
# Stemming
# stemming is the process of reducing inflected words to their root word
# Stemming is a natural language processing technique that is used to reduce words to their base form, also known as the root form


In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stemming(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
def stemming(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
stemming("dancing dance dances danced")

In [ ]:
# Lemmatization
import nltk
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()

In [ ]:
lm.lemmatize("walking",pos='v')

In [ ]:
new_df['tags']=new_df['tags'].apply(stemming)

In [ ]:
vectors=cv.fit_transform(new_df['tags'])

In [ ]:
cv.get_feature_names_out()[0:200]

In [ ]:
# cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
similarity

In [ ]:
sorted(list(enumerate(similarity[ 0])),key=lambda x:x[1],reverse=True)[1:11]

In [ ]:
def recommend(movie):
  index=new_df[new_df['title']==movie].index[0]
  distances=sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)
  for i in distances[1:11]:
    print(new_df.iloc[i[0]].title)

In [ ]:
new_df[new_df['title']=="Alice in Wonderland"].index[0]

In [ ]:
recommend("Iron Man")

In [ ]:
!pip install streamlit --quiet

In [ ]:
import pickle


In [ ]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import requests

movies=pickle.load(open('movie_list.pkl','rb'))
similarity=pickle.load(open('similarity.pkl','rb'))

def recommend(movie):
  index=movies[movies['title']==movie].index[0]
  distances=sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)
  recommended_movie_names=[]
  for i in distances[1:11]:
    recommended_movie_names.append(movies.iloc[i[0]].title)
  return recommended_movie_names

st.header("Movie Recommender System")
movie_list=movies['title'].values
selected_movie=st.selectbox("Type or select a movie from the list",movie_list)

if st.button("Show Recommendations"):
  recommended_movie_names=recommend(selected_movie)
  c1,c2,c3=st.columns(3)
  with c1:
    st.text(recommended_movie_names[0])
  with c2:
    st.text(recommended_movie_names[1])
  with c3:
    st.text(recommended_movie_names[2])


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.68.28:8501

